In [21]:
# Import required modules
import cv2
import numpy as np
import os
import glob


# Define the dimensions of checkerboard
CHECKERBOARD = (6, 6)


# stop the iteration when specified
# accuracy, epsilon, is reached or
# specified number of iterations are completed.
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Vector for 3D points
threedpoints = []

# Vector for 2D points
twodpoints = []


# 3D points real world coordinates
objectp3d = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objectp3d[0, :, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

# Extracting path of individual image stored
# in a given directory. Since no path is
# specified, it will take current directory
# jpg files alone
images = glob.glob('Calibration_images/Sag/*.jpg')

for filename in images:
    image = cv2.imread(filename)
    grayColor = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    # If desired number of corners are
    # found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(grayColor, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

    # If desired number of corners can be detected then,
    # refine the pixel coordinates and display
    # them on the images of checker board
    if ret == True:
        threedpoints.append(objectp3d)

        # Refining pixel coordinates
        # for given 2d points.
        corners2 = cv2.cornerSubPix(
            grayColor, corners, (11, 11), (-1, -1), criteria)

        twodpoints.append(corners2)

        # Draw and display the corners
        image = cv2.drawChessboardCorners(image, CHECKERBOARD, corners2, ret)

    cv2.imshow('img', image)
    cv2.waitKey(0)

cv2.destroyAllWindows()

h, w = image.shape[:2]


# Perform camera calibration by
# passing the value of above found out 3D points (threedpoints)
# and its corresponding pixel coordinates of the
# detected corners (twodpoints)
ret, matrix, distortion, r_vecs, t_vecs = cv2.calibrateCamera(threedpoints, twodpoints, grayColor.shape[::-1], None, None)

newcameramtx, roi = cv2.getOptimalNewCameraMatrix(matrix, distortion, (w,h), 1, (w,h))

# Displaying required output
print(" Camera matrix:")
print(matrix)

print("\n Distortion coefficient:")
print(distortion)

#print("\n Rotation Vectors:")
#print(r_vecs)

#print("\n Translation Vectors:")
#print(t_vecs)


 Camera matrix:
[[1.57212061e+03 0.00000000e+00 6.50718376e+02]
 [0.00000000e+00 1.56728861e+03 3.82149599e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

 Distortion coefficient:
[[-0.35596872  0.45351657 -0.01160594 -0.00270191 -1.30458024]]


In [20]:
images = glob.glob('Images_to_undistort/*/Cor/*.jpg')
for filename in images:
    img = cv2.imread(filename)
    h,  w = img.shape[:2]

    # undistort
    mapx, mapy = cv2.initUndistortRectifyMap(matrix, distortion, None, newcameramtx, (w,h), 5)
    dst = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)
    # crop the image
    #x, y, w, h = roi
    #dst = dst[y:y+h, x:x+w]
    output_filename = filename[:-4] + '_undistorted.jpg'
    cv2.imwrite(output_filename, dst)

### Below were tests to figure out how to get the code running

In [7]:
img = cv2.imread('Standing Left Side.jpg')
h,  w = img.shape[:2]

# undistort
dst = cv2.undistort(img, matrix, distortion, None, newcameramtx)
# crop the image
#x, y, w, h = roi
#dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult1.png', dst)

True

True

In [6]:
import glob
glob.glob('Images_to_undistort/*/Cor/*.jpg')

['Images_to_undistort\\Allen, Steven RL1103260\\Cor\\Standing Cor Away.jpg',
 'Images_to_undistort\\Allen, Steven RL1103260\\Cor\\Standing Cor Twrd (2).jpg',
 'Images_to_undistort\\Allen, Steven RL1103260\\Cor\\Standing Cor Twrd.jpg',
 'Images_to_undistort\\Barton, Mark RL1181385\\Cor\\Stanging Cor Away.jpg',
 'Images_to_undistort\\Barton, Mark RL1181385\\Cor\\Stanging Cor Twrd (2).jpg',
 'Images_to_undistort\\Barton, Mark RL1181385\\Cor\\Stanging Cor Twrd (3).jpg',
 'Images_to_undistort\\Barton, Mark RL1181385\\Cor\\Stanging Cor Twrd.jpg',
 'Images_to_undistort\\Barton, Mark RL1181385\\Cor\\Stanging Left Side (3).jpg',
 'Images_to_undistort\\Barton, Mark RL1181385\\Cor\\Stanging Right Side.jpg',
 'Images_to_undistort\\Bell, William RL1250832\\Cor\\3iq0102.2107274.20211102144547-0.00.01.02.jpg',
 'Images_to_undistort\\Bell, William RL1250832\\Cor\\3iq0114.2107274.20211102145556-0.00.00.96.jpg',
 'Images_to_undistort\\Bell, William RL1250832\\Cor\\3iq0114.2107274.20211102145556-0.00.16.